In [92]:
import pandas as pd
import aqi
# se combinan datos de metereologia y particulas contaminantes y se calcula icap en base a pm10

In [93]:
d_pm25 = pd.read_csv('../data/processed/pm_2.5.csv',sep=';', thousands=None, decimal=',')
d_pm10 = pd.read_csv('../data/processed/pm_10.csv',sep=';', thousands=None, decimal=',')

Metereologia = pd.read_csv('../data/processed/Metereologia.csv',sep=';', thousands=None, decimal=',')


In [94]:
d_contaminantes = pd.merge(d_pm25, d_pm10, on='fecha', how='outer')

nuevos_nombres = {
    'registro_x': 'pm2.5',
    'registro_y': 'pm10'
}
d_contaminantes = d_contaminantes.rename(columns=nuevos_nombres)
# Eliminando la fila con índice 0
d_contaminantes.drop(index=0, inplace=True)
d_contaminantes.reset_index(drop=True, inplace=True)

In [95]:
ruta_csv = 'C:/Users/benja/Documents/Particulas_contaminantes_chillan/data/raw/datos_direccion_meteorologica/precipitaciones/datos_concatenados_procesados.csv'


lluvias = pd.read_csv(ruta_csv)

lluvias['Fecha'] = pd.to_datetime(lluvias['Fecha'], errors='coerce')
lluvias = lluvias.dropna(subset=['Fecha'])
lluvias.columns = ['fecha', 'precipitacion']
fecha_inicio = '2013-01-01'
fecha_fin = '2023-03-31'

lluvias_filtradas = lluvias[(lluvias['fecha'] >= fecha_inicio) & (lluvias['fecha'] <= fecha_fin)]


lluvias_filtradas

,fecha,precipitacion
0,2013-01-01,0
1,2013-01-02,0
2,2013-01-03,0
3,2013-01-04,0
4,2013-01-05,0.0
...,...,...
3808,2023-03-27,0
3809,2023-03-28,0.1
3810,2023-03-29,0
3811,2023-03-30,0


In [96]:
Metereologia.rename(columns={'ff_Valor': 'viento'}, inplace=True)
Metereologia.rename(columns={'dd_Valor': 'dir_viento'}, inplace=True)

# DataSet = pd.merge(d_contaminantes, Metereologia, on='fecha', how='outer')

# Convertir la columna 'fecha' a datetime64[ns] para cada DataFrame
d_contaminantes['fecha'] = pd.to_datetime(d_contaminantes['fecha'])
Metereologia['fecha'] = pd.to_datetime(Metereologia['fecha'])

# Ahora, realiza el merge
DataSet = pd.merge(d_contaminantes, Metereologia, on='fecha', how='outer')



# Convertir la columna 'fecha' del tercer DataFrame a datetime64[ns]
lluvias_filtradas['fecha'] = pd.to_datetime(lluvias_filtradas['fecha'])

# Merge del DataSet con el tercer DataFrame
DataSet = pd.merge(DataSet, lluvias_filtradas, on='fecha', how='outer')

DataSet.head()


C:\Users\benja\AppData\Local\Temp\ipykernel_5472\831213413.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lluvias_filtradas['fecha'] = pd.to_datetime(lluvias_filtradas['fecha'])


,fecha,pm2.5,pm10,humedad,puntoRocio,temperatura,dir_viento,viento,precipitacion
0,2013-01-01,9.3612,25.2396,44.8462,10.0615,25.0308,174.6154,3.7692,0
1,2013-01-02,13.7865,36.1072,43.3077,12.1000,25.8000,256.1538,3.7692,0
2,2013-01-03,12.2568,29.2324,62.8462,13.4000,21.2462,213.0769,2.7692,0
3,2013-01-04,12.6945,33.1919,68.6923,13.5385,19.6000,154.6154,2.7692,0
4,2013-01-05,8.6662,26.3441,53.5385,12.3231,23.3231,235.3846,3.9231,0.0


In [97]:
def AQI(row):
    pm25 = row['pm2.5']
    pm10 = row['pm10']
    
    try:
        myaqi = aqi.to_aqi([
            (aqi.POLLUTANT_PM25, pm25),
            (aqi.POLLUTANT_PM10, pm10)
        ])
    except IndexError:
        myaqi = 500

    return myaqi

DataSet['AQI'] = DataSet.apply(AQI, axis=1)


In [98]:
# ETIQUETACION DE CALIDAD DEL AIRE BASADO EN ICAP
# ICAP            MP10 µg/m3N (24 horas)
# 0                0
# 100              150
# 500              330


# def ICAP(PM10):

#     if PM10 < 0:
#         raise ValueError("PM10 debe ser mayor igual a 0")
    

#     if 0 <= PM10 <= 150:
#         ICAP = PM10 / 1.5
#     elif 150 <PM10 :
#         ICAP = (PM10 - 114) / 0.36

    
#     return ICAP

# DataSet['icap'] = DataSet['pm10'].apply(ICAP)



In [99]:
# Definir los bordes de los bins (contenedores) para los niveles de calidad del aire
bins = [0, 50, 100, 150, 200, 300, float('inf')]

# Definir las etiquetas para los niveles de calidad del aire
labels = ['Bueno', 'Moderado', 'Regular', 'Malo', 'Muy Malo', 'Peligroso']

# Crear la nueva columna 'calidad_aire' usando pd.cut
DataSet['calidad_aire'] = pd.cut(DataSet['AQI'], bins=bins, labels=labels, right=False)  # right=False para que los intervalos sean semi-abiertos a la izquierda

# Verificar el conteo de valores en la nueva columna
conteo_calidad_aire = DataSet['calidad_aire'].value_counts()

In [100]:
conteo_calidad_aire = DataSet['calidad_aire'].value_counts()
print(conteo_calidad_aire)

calidad_aire
Bueno        1535
Moderado     1341
Malo          436
Regular       391
Muy Malo       34
Peligroso       5
Name: count, dtype: int64


In [101]:
nombre = 'Dataset_Particulas_contaminantes'

path = '../data/processed/{}.csv'.format(nombre)

DataSet['precipitacion'] = pd.to_numeric(DataSet['precipitacion'])

DataSet.to_csv(path, sep=';',index=False, float_format='%.4f', decimal=',')
DataSet

,fecha,pm2.5,pm10,humedad,puntoRocio,temperatura,dir_viento,viento,precipitacion,AQI,calidad_aire
0,2013-01-01,9.3612,25.2396,44.8462,10.0615,25.0308,174.6154,3.7692,0.0,39,Bueno
1,2013-01-02,13.7865,36.1072,43.3077,12.1000,25.8000,256.1538,3.7692,0.0,54,Moderado
2,2013-01-03,12.2568,29.2324,62.8462,13.4000,21.2462,213.0769,2.7692,0.0,51,Moderado
3,2013-01-04,12.6945,33.1919,68.6923,13.5385,19.6000,154.6154,2.7692,0.0,52,Moderado
4,2013-01-05,8.6662,26.3441,53.5385,12.3231,23.3231,235.3846,3.9231,0.0,36,Bueno
...,...,...,...,...,...,...,...,...,...,...,...
3737,2023-03-27,18.5833,47.5000,53.8750,5.0792,15.8042,180.1250,3.7083,0.0,64,Moderado
3738,2023-03-28,5.8125,23.0000,78.2500,8.4962,12.2885,155.5000,4.3846,0.1,24,Bueno
3739,2023-03-29,5.3542,21.5000,63.3750,5.4625,13.4583,219.5000,6.9167,0.0,22,Bueno
3740,2023-03-30,10.3333,32.0000,55.0417,2.6625,12.6625,216.6667,8.7083,0.0,43,Bueno
